# 하이브리드 검색(Hybrid Search)

---

## 1. 개념 이해

### 1.1 검색 방식 분류

#### 🔍 **의미론적 검색 (Semantic Search)**
- **정의**: 벡터 임베딩을 활용한 의미 기반 검색
- **특징**: 
  - 동의어와 문맥적 의미를 파악
  - 자연어 질의에 효과적
  - 유사도 기반 매칭
- **장점**: 의미적 연관성이 높은 문서 검색
- **단점**: 정확한 키워드 매칭 약함

#### 🔍 **키워드 검색 (Keyword Search)**
- **정의**: BM25 알고리즘 기반 키워드 매칭 검색
- **특징**:
  - 직접적인 단어/구문 매칭
  - 계산 효율성 높음
  - 전통적인 정보검색 방식
- **장점**: 정확한 키워드 매칭, 빠른 처리 속도
- **단점**: 의미적 유사성 파악 제한적

#### 🔍 **하이브리드 검색 (Hybrid Search)**
- **정의**: 키워드 검색과 의미론적 검색의 결합
- **특징**:
  - EnsembleRetriever를 통한 구현
  - 가중치 조정으로 성능 최적화
  - 두 검색 방식의 시너지 효과
- **장점**: 포괄적이고 정확한 검색 결과
- **단점**: 구현 복잡도 증가, 계산 비용 상승

### 1.2 BM25 알고리즘 이해

- BM25는 TF-IDF의 확장된 버전으로, 다음 공식을 사용합니다:

  ```markdown
  BM25(qi, D) = IDF(qi) × (f(qi, D) × (k1 + 1)) / (f(qi, D) + k1 × (1 - b + b × |D| / avgdl))
  ```

  - `f(qi, D)`: 문서 D에서 단어 qi의 빈도
  - `|D|`: 문서 D의 길이
  - `avgdl`: 평균 문서 길이
  - `k1, b`: 조정 매개변수

---

## 2. 환경 설정

### 2.1 라이브러리 설치

```bash
uv pip install ranx-k
```

### 2.2 환경 변수 설정

In [ ]:
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

### 2.3 기본 라이브러리

In [ ]:
import os
import json
from glob import glob
from pprint import pprint
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

# # 한글 폰트 인식 - Windows
# import matplotlib 
# font_name = matplotlib.font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
# matplotlib.rc('font', family=font_name)

# 한글 폰트 인식 - Mac
import matplotlib
matplotlib.rc('font', family='AppleGothic')

# 마이너스 부호 인식
matplotlib.rc("axes", unicode_minus = False)

# LangChain 핵심
from langchain_core.documents import Document
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# 데이터 처리
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 평가
import ranx_k

---

## 3. 데이터 준비

### 3.1 텍스트 데이터 로드

In [ ]:
def load_text_files(file_patterns):

    documents = []
    
    for pattern in file_patterns:
        files = glob(pattern)
        for file_path in files:
            try:
                loader = TextLoader(file_path, encoding='utf-8')
                docs = loader.load()
                documents.extend(docs)
                print(f"✅ {file_path} 로드 완료")
            except Exception as e:
                print(f"❌ {file_path} 로드 실패: {e}")
    
    return documents

# 로드할 문서의 패턴 정의
# 예시: 'data/*.txt', 'data/*.json' 등
# 현재는 'data/*_KR.md' 패턴만 사용
file_patterns = [
    'data/*_KR.md',
    # 'data/*.txt',
    # 'data/*.json'
]
raw_documents = load_text_files(file_patterns)
print(f"총 {len(raw_documents)}개 문서 로드됨")

### 3.2 문서 분할 및 전처리

In [ ]:
def preprocess_documents(documents, company_mapping=None):
    """
    문서 전처리 및 메타데이터 추가
    
    Args:
        documents (list): 원본 문서 리스트
        company_mapping (dict): 회사명 매핑 정보
    
    Returns:
        list: 전처리된 Document 객체 리스트
    """
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        encoding_name="cl100k_base",
        separators=['\n\n', '\n', r'(?<=[.!?])\s+'],
        chunk_size=300,
        chunk_overlap=50,
        is_separator_regex=True,
        keep_separator=True,
    )
    chunks = text_splitter.split_documents(documents)
    
    processed_docs = []
    for chunk in chunks:
        # 메타데이터 추가
        source_file = chunk.metadata.get('source', '')
        
        # 회사명 추출
        if company_mapping:
            company = 'unknown'
            for keyword, name in company_mapping.items():
                if keyword in source_file.lower():
                    company = name
                    break
        else:
            company = 'default'
        
        # Document 객체 생성
        doc = Document(
            page_content=f"<Document>\n{chunk.page_content}\n</Document>\n<Source>이 문서는 '{company}'에 대한 문서입니다.</Source>",
            metadata={
                **chunk.metadata,
                'company': company,
                'language': 'ko',
                'chunk_length': len(chunk.page_content)
            }
        )
        processed_docs.append(doc)
    
    return processed_docs

# 회사명 매핑 정보
# 예시: 'tesla' -> '테슬라', 'rivian' -> '리비안'
# 실제 사용 시에는 더 많은 회사명을 추가할 수 있음
company_mapping = {
    '테슬라': '테슬라(tesla)',
    '리비안': '리비안(rivian)',
}

processed_docs = preprocess_documents(raw_documents, company_mapping)
print(f"총 {len(processed_docs)}개 청크 생성됨")

In [ ]:
# 결과 확인
for i, doc in enumerate(processed_docs[:3]+ processed_docs[-3:]):
    print(f"\n[청크 {i+1}]")
    print(f"회사: {doc.metadata['company']}")
    print(f"내용: {doc.page_content}")
    print("-" * 50)

---

## 4. 검색 방법론

### 4.1 의미론적 검색 (Semantic Search)

#### 4.1.1 벡터 저장소 생성

In [ ]:
import os
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

def create_or_load_vector_store(documents, collection_name="hybrid_search_db"):
    """
    Chroma 벡터 저장소 생성 (기존 컬렉션이 있으면 로드하고, 없으면 새로 생성)
    
    Args:
        documents (list): Document 객체 리스트
        collection_name (str): 컬렉션 이름
    
    Returns:
        Chroma: 벡터 저장소 객체

    """
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    persist_directory = "./local_chroma_db"
    
    # 디렉토리와 컬렉션 파일이 존재하는지 확인
    if os.path.exists(persist_directory):
        try:
            # 기존 컬렉션 로드 시도
            vector_store = Chroma(
                collection_name=collection_name,
                embedding_function=embeddings,
                persist_directory=persist_directory
            )
            
            # 컬렉션에 문서가 있는지 확인
            if vector_store._collection.count() > 0:
                print(f"✅ 기존 벡터 저장소 로드 완료: {vector_store._collection.count()}개 문서")
                return vector_store
        except Exception as e:
            print(f"기존 컬렉션 로드 실패: {e}")
    
    # 새로운 벡터 저장소 생성
    vector_store = Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        collection_name=collection_name,
        persist_directory=persist_directory,
        collection_metadata={'hnsw:space': 'cosine'}
    )
    print(f"✅ 새 벡터 저장소 생성 완료: {vector_store._collection.count()}개 문서")
    return vector_store

# 벡터 저장소 생성
vector_store = create_or_load_vector_store(processed_docs)

#### 4.1.2 의미론적 검색 실행

In [ ]:
def semantic_search(vector_store, query, k=5):
    """
    의미론적 검색 실행
    
    Args:
        vector_store: Chroma 벡터 저장소
        query (str): 검색 쿼리
        k (int): 반환할 문서 수
    
    Returns:
        list: 검색된 Document 객체 리스트
    """
    # 검색기 생성
    retriever = vector_store.as_retriever(search_kwargs={"k": k})
    
    # 검색 실행
    results = retriever.invoke(query)
        
    return results

# 검색 테스트
query = "리비안은 언제 설립되었나요?"
semantic_results = semantic_search(vector_store, query, k=3)

print(f"Query: {query}")
print(f"검색 결과 ({len(semantic_results)}개)")
for i, doc in enumerate(semantic_results, 1):
    print(f"\n[{i}] 회사: {doc.metadata.get('company', 'N/A')}")
    print(f"내용:\n{doc.page_content}")
    print("-" * 50)


### 4.2 키워드 검색 (Keyword Search)

#### 4.2.1 한국어 토크나이저 설정

In [ ]:
def setup_korean_tokenizer():
    """
    한국어 토크나이저 설정
    
    Returns:
        Kiwi: 한국어 토크나이저 객체
    """
    from kiwipiepy import Kiwi
    kiwi = Kiwi()
    
    # 사용자 정의 단어 추가
    custom_words = [
        ('리비안', 'NNP'),  # 고유명사
        ('테슬라', 'NNP'),  # 고유명사
        ('전기차', 'NNG'),  # 일반명사
    ]
    
    for word, pos in custom_words:
        kiwi.add_user_word(word, pos)
        print(f"✅ 단어 추가: {word} ({pos})")
    
    return kiwi

def korean_tokenizer(text, kiwi_model):
    """
    한국어 토크나이저 함수
    
    Args:
        text (str): 토큰화할 텍스트
        kiwi_model: Kiwi 모델 객체
    
    Returns:
        list: 토큰 리스트
    """
    return [token.form for token in kiwi_model.tokenize(text)]

# 토크나이저 설정
kiwi_model = setup_korean_tokenizer()

# 토큰화 테스트
test_text = "리비안은 언제 설립되었나요?"
tokens = korean_tokenizer(test_text, kiwi_model)
print(f"원문: {test_text}")
print(f"토큰: {tokens}")

#### 4.2.2 BM25 검색기 생성
- uv pip install rank_bm25

In [ ]:
# BM25 검색기 생성 (한국어 토크나이저 미적용)
bm25_retriever = BM25Retriever.from_documents(
    documents=processed_docs,
    k=5
)

# 검색 테스트
keyword_results = bm25_retriever.invoke(query)

print(f"Query: {query}")
print(f"🔍 검색 결과 ({len(keyword_results)}개)")
for i, doc in enumerate(keyword_results, 1):
    print(f"\n[{i}] 회사: {doc.metadata.get('company', 'N/A')}")
    print(f"내용:\n{doc.page_content}")
    print("-" * 50)


In [ ]:
def create_bm25_retriever(documents, kiwi_model, k=5):
    """
    BM25 검색기 생성 (한국어 토크나이저 적용)
    
    Args:
        documents (list): Document 객체 리스트
        kiwi_model: Kiwi 토크나이저
        k (int): 반환할 문서 수
    
    Returns:
        BM25Retriever: BM25 검색기 객체
    """
    
    # 한국어 토크나이저 적용
    def preprocess_func(text):
        return korean_tokenizer(text, kiwi_model)
    
    # BM25 검색기 생성
    bm25_retriever = BM25Retriever.from_documents(
        documents=documents,
        preprocess_func=preprocess_func,
        k=k
    )
    
    print(f"✅ BM25 검색기 생성 완료: {len(documents)}개 문서 인덱싱")
    return bm25_retriever

# BM25 검색기 생성
bm25_retriever = create_bm25_retriever(processed_docs, kiwi_model, k=5)

In [ ]:
# 검색 테스트
keyword_results = bm25_retriever.invoke(query)

print(f"Query: {query}")
print(f"🔍 검색 결과 ({len(keyword_results)}개)")
for i, doc in enumerate(keyword_results, 1):
    print(f"\n[{i}] 회사: {doc.metadata.get('company', 'N/A')}")
    print(f"내용:\n{doc.page_content}")
    print("-" * 50)


#### 4.2.3 BM25 점수 확인

In [ ]:
def analyze_bm25_scores(bm25_retriever, query, kiwi_model, top_k=5):
    """
    BM25 점수 분석
    
    Args:
        bm25_retriever: BM25 검색기
        query (str): 검색 쿼리
        kiwi_model: Kiwi 토크나이저
        top_k (int): 상위 k개 결과
    """
    # 쿼리 토큰화
    tokenized_query = korean_tokenizer(query, kiwi_model)
    print(f"쿼리 토큰: {tokenized_query}")
    
    # BM25 점수 계산
    doc_scores = bm25_retriever.vectorizer.get_scores(tokenized_query)
    
    # 점수 정렬
    doc_scores_sorted = sorted(
        enumerate(doc_scores), 
        key=lambda x: x[1], 
        reverse=True
    )
    
    print(f"\n📊 상위 {top_k}개 문서의 BM25 점수:")
    print("-" * 80)
    
    for rank, (idx, score) in enumerate(doc_scores_sorted[:top_k], 1):
        doc = bm25_retriever.docs[idx]
        print(f"[{rank}] 점수: {score:.4f}")
        print(f"    회사: {doc.metadata.get('company', 'N/A')}")
        print(f"    내용: {doc.page_content[:200]}...")
        print("-" * 80)

# BM25 점수 분석
analyze_bm25_scores(bm25_retriever, query, kiwi_model)

### 4.3 하이브리드 검색 (Hybrid Search)

In [ ]:
def create_hybrid_retriever(vector_store, bm25_retriever, weights=None):
    """
    하이브리드 검색기 생성
    
    Args:
        vector_store: 벡터 저장소
        bm25_retriever: BM25 검색기
        weights (list): 가중치 [의미론적, 키워드]
    
    Returns:
        EnsembleRetriever: 하이브리드 검색기
    """
    if weights is None:
        weights = [0.5, 0.5]  # 기본값: 동일한 가중치
    
    # 의미론적 검색기 생성
    semantic_retriever = vector_store.as_retriever(search_kwargs={"k": 5})
    
    # 앙상블 검색기 생성
    ensemble_retriever = EnsembleRetriever(
        retrievers=[semantic_retriever, bm25_retriever],
        weights=weights
    )
    
    print(f"✅ 하이브리드 검색기 생성 완료")
    print(f"   가중치: 의미론적 {weights[0]}, 키워드 {weights[1]}")
    
    return ensemble_retriever

# 하이브리드 검색기 생성
hybrid_retriever = create_hybrid_retriever(vector_store, bm25_retriever)

In [ ]:
# 검색 테스트
hybrid_results = hybrid_retriever.invoke(query)

print(f"Query: {query}")
print(f"🔍 검색 결과 ({len(hybrid_results)}개)")
for i, doc in enumerate(hybrid_results, 1):
    print(f"\n[{i}] 회사: {doc.metadata.get('company', 'N/A')}")
    print(f"내용:\n{doc.page_content}")
    print("-" * 50)


---

## 5. 평가 및 비교

### 5.1 평가 데이터셋 준비

In [ ]:
def load_evaluation_dataset(file_path):
    """
    평가 데이터셋 로드
    
    Args:
        file_path (str): 평가 데이터 파일 경로
    
    Returns:
        pandas.DataFrame: 평가 데이터셋
    """
    try:
        if file_path.endswith('.xlsx'):
            df = pd.read_excel(file_path)
        elif file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        else:
            raise ValueError("지원하지 않는 파일 형식")
        
        print(f"✅ 평가 데이터셋 로드: {len(df)}개 질문")
        return df
    except Exception as e:
        print(f"❌ 데이터셋 로드 실패: {e}")
        return None

# 평가 데이터셋 로드

eval_df = load_evaluation_dataset("./data/synthetic_testset.csv")
eval_df.head(3)

In [ ]:
def prepare_evaluation_data(df):
    """
    평가 데이터 전처리
    
    Args:
        df (pandas.DataFrame): 원본 데이터프레임
    
    Returns:
        tuple: (질문 리스트, 정답 문서 리스트)
    """
    questions = df['user_input'].tolist()
    
    # 정답 문서 파싱
    reference_contexts = []
    for contexts in df['reference_contexts']:
        if isinstance(contexts, str):
            # 문자열을 리스트로 변환
            context_list = eval(contexts)
        else:
            context_list = contexts
        
        # Document 객체로 변환
        docs = [Document(page_content=ctx) for ctx in context_list]
        reference_contexts.append(docs)
    
    return questions, reference_contexts

# 평가 데이터 전처리
questions, reference_contexts = prepare_evaluation_data(eval_df)

# 평가 데이터 확인
for i, (q, refs) in enumerate(zip(questions[:3], reference_contexts[:3])):
    print(f"\n[질문 {i+1}]")
    print(f"질문: {q}")
    print(f"정답 문서: {len(refs)}개")
    for j, ref in enumerate(refs):
        print(f"  [{j+1}] 내용: {ref.page_content[:50]}...")  # 내용 일부만 출력
    print("-" * 50)

# 평가 데이터 확인
for i, (q, refs) in enumerate(zip(questions[-3:], reference_contexts[-3:])):
    print(f"\n[질문 {i+1}]")
    print(f"질문: {q}")
    print(f"정답 문서: {len(refs)}개")
    for j, ref in enumerate(refs):
        print(f"  [{j+1}] 내용: {ref.page_content[:50]}...")  # 내용 일부만 출력
    print("-" * 50)


### 5.2 ranx-k 라이브러리 활용한 평가

- ROUGE 점수 기반 평가 
- 아이디어: 텍스트 오버랩을 통한 직접적 유사도 측정

- **ranx-k 설치 방법**

    ```bash
    # ranx-k 라이브러리 설치
    uv pip install ranx-k
    ```

- **ranx 이슈 사항**

    - **ID 매칭 의존성**: 정확한 문서 ID 일치가 필요
    - **의미적 유사도 무시**: 내용이 비슷해도 ID가 다르면 0점
    - **청킹 방식 변화 대응 불가**: 청크 크기나 방식이 바뀌면 평가 불가능
    - **실제 사용 환경과 괴리**: 실제로는 의미적 관련성이 중요

In [ ]:
from ranx_k.evaluation import evaluate_with_ranx_similarity

# ranx-k 평가 실행 (rouge 점수가 높은 경우) -> 문자열 유사도 기반 평가
ranx_k_results = evaluate_with_ranx_similarity(
    retriever=hybrid_retriever,
    questions=questions,
    reference_contexts=reference_contexts,
    k=5,
    method='kiwi_rouge',  
    similarity_threshold=0.8,
)

In [ ]:
# ranx-k 평가 실행 (embedding 점수가 높은 경우) -> 의미적 유사도 기반 평가
ranx_k_results = evaluate_with_ranx_similarity(
    retriever=hybrid_retriever,
    questions=questions,
    reference_contexts=reference_contexts,
    k=5,
    method='embedding',  
    embedding_model="BAAI/bge-m3",
    similarity_threshold=0.9
)

### 5.4 검색 방법 비교

In [ ]:
def compare_retrieval_methods(vector_store, bm25_retriever, questions, reference_contexts, k=5):
    """
    다양한 검색 방법 성능 비교
    
    Args:
        vector_store: 벡터 저장소
        bm25_retriever: BM25 검색기
        questions (list): 질문 리스트
        reference_contexts (list): 정답 문서 리스트
        k (int): 평가할 상위 k개 결과
    
    Returns:
        pandas.DataFrame: 비교 결과
    """
    results = []
    
    # 검색 방법들 정의
    bm25_retriever.k = k
    retrievers = {
        "의미론적 검색": vector_store.as_retriever(search_kwargs={"k": k}),
        "키워드 검색": bm25_retriever,
        "하이브리드 (5:5)": create_hybrid_retriever(vector_store, bm25_retriever, [0.5, 0.5]),
        "하이브리드 (7:3)": create_hybrid_retriever(vector_store, bm25_retriever, [0.7, 0.3]),
        "하이브리드 (3:7)": create_hybrid_retriever(vector_store, bm25_retriever, [0.3, 0.7]),
    }
    
    for method_name, retriever in retrievers.items():
        print(f"🔄 {method_name} 평가 중...")
        
        # ranx-k 평가 실행 (rouge 점수가 높은 경우) -> 문자열 유사도 기반 평가
        ranx_k_results = evaluate_with_ranx_similarity(
            retriever=retriever,
            questions=questions,
            reference_contexts=reference_contexts,
            k=k,
            method='kiwi_rouge',  
            similarity_threshold=0.8,
        )
        
        # 평가 결과 정리 
        result = {
            "method": method_name,
            "hit_rate": ranx_k_results.get("hit_rate@5", 0),
            "ndcg": ranx_k_results.get("ndcg@5", 0),
            "map": ranx_k_results.get("map@5", 0),
            "mrr": ranx_k_results.get("mrr", 0),
        }

        results.append(result)

    # 결과 DataFrame 생성
    results_df = pd.DataFrame(results)
    results_df.set_index("method", inplace=True)
    results_df.sort_values(by="hit_rate", ascending=False, inplace=True)

    return results_df


# 검색 방법 성능 비교
comparison_results = compare_retrieval_methods(vector_store, bm25_retriever, questions, reference_contexts, k=5)

In [ ]:
comparison_results

### 5.5 시각화

In [ ]:
def visualize_comparison_results(comparison_df):
    """
    검색 방법 비교 결과 시각화
    
    Args:
        comparison_df (pandas.DataFrame): 비교 결과 데이터프레임

        result = {
            "method": method_name,
            "hit_rate": ranx_k_results.get("hit_rate@5", 0),
            "ndcg": ranx_k_results.get("ndcg@5", 0),
            "map": ranx_k_results.get("map@5", 0),
            "mrr": ranx_k_results.get("mrr", 0),
        }

    """
    
    # 그래프 생성
    fig, ax = plt.subplots(1, 4, figsize=(12, 8))
    sns.barplot(x=comparison_df.index, y='hit_rate', data=comparison_df, ax=ax[0])
    ax[0].set_title('Hit Rate@5')
    ax[0].set_ylabel('Hit Rate')
    ax[0].set_xlabel('Retrieval Method')        
    ax[0].tick_params(axis='x', rotation=45)

    sns.barplot(x=comparison_df.index, y='ndcg', data=comparison_df, ax=ax[1])
    ax[1].set_title('NDCG@5')
    ax[1].set_ylabel('NDCG')
    ax[1].set_xlabel('Retrieval Method')
    ax[1].tick_params(axis='x', rotation=45)                

    sns.barplot(x=comparison_df.index, y='map', data=comparison_df, ax=ax[2])
    ax[2].set_title('MAP@5')
    ax[2].set_ylabel('MAP')         
    ax[2].set_xlabel('Retrieval Method')
    ax[2].tick_params(axis='x', rotation=45)    

    sns.barplot(x=comparison_df.index, y='mrr', data=comparison_df, ax=ax[3])
    ax[3].set_title('MRR')
    ax[3].set_ylabel('MRR')
    ax[3].set_xlabel('Retrieval Method')
    ax[3].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

    # 그래프 출력
    plt.savefig("retrieval_comparison_results.png")

# 검색 방법 비교 결과 시각화
visualize_comparison_results(comparison_results)

---

## 6. 실습 문제

**목표**: 기본적인 하이브리드 검색 시스템을 구축하고 테스트해보세요.

**문제**:
1. 제공된 텍스트 데이터를 사용하여 벡터 저장소와 BM25 검색기를 생성하세요.
2. 동일한 가중치(0.5, 0.5)로 하이브리드 검색기를 만드세요.
3. 다음 질문들에 대해 검색을 수행하고 결과를 비교하세요:
   - "테슬라의 전기차 모델은 어떤 것들이 있나요?"
   - "리비안의 경쟁력은 무엇인가요?"
   - "테슬라와 리비안이 경쟁하는 분야는 무엇인가요?"

In [ ]:
# 여기에 코드를 작성하세요

# 1. 데이터 로드 및 전처리

# 2. 벡터 저장소 생성

# 3. BM25 검색기 생성

# 4. 하이브리드 검색기 생성

# 5. 검색 테스트
queries = [
    "테슬라의 전기차 모델은 어떤 것들이 있나요?",
    "리비안의 경쟁력은 무엇인가요?",
    "테슬라와 리비안이 경쟁하는 분야는 무엇인가요?"
]

for query in queries:
    print(f"질문: {query}")
    # 검색 수행 및 결과 출력
    pass